In [1]:
import pandas as pd
import numpy as np
import os
import glob, re
from datetime import datetime
# import personal packages
import sys
src_dir = os.path.join(os.path.dirname(os.getcwd()),'src')
sys.path.append(src_dir)

In [2]:
# import local packages
from d01_data.read_excel_file import read_excel_file
from tools.assortment_creation import assortment_creation
from tools.longest_streak_counter import longest_streak_counter

In [3]:
date = datetime.now().strftime("%m%d%Y_%H%M")
date

'08142023_1622'

# Load Data

In [4]:
df_dir = read_excel_file('01_raw', '*merchandise*')

In [5]:
df = pd.read_excel(df_dir)

# Clean data

Gender data is not uniformed

In [6]:
# copy data frame to keep original data
df_clean = df.copy()

In [7]:
# Dictionaries to replace values / normalized categories
replace_dict = {"MEN'S":'MEN',
"GIRL'S":'GIRL',
"JUNIOR'S":'JUNIOR',
"  WOMEN":'WOMEN',
"  UNISEX":'UNISEX',
"  MEN":'MEN',
"  CHILD UNISEX":'CHILD UNISEX',
" WOMEN":'WOMEN',
"BOY'S":'BOY',
"Women":'WOMEN',
"WOMEN PETITE":'WOMEN',
"UNISEX                  MENWOMEN":'UNISEX',
"Unisex":'UNISEX',
"Men":'MEN',
"Girl":'GIRL',
'UNISEX                        (MEN WOMEN':'UNISEX',
'UNISEX                        MEN WOMEN':'UNISEX',
'UNISEX SHORT               (MEN WOMEN ME':'UNISEX'}

In [8]:
df_clean.loc[:,'Gender'] = df_clean.loc[:,'Gender'].replace(replace_dict)

# EDA

In [9]:
df_clean.loc[:,'Gender'].value_counts()

WOMEN             229912
UNISEX             22249
MEN                20001
                   15009
WOMEN'S             2135
CHILD UNISEX         943
JUNIOR               301
MISSY                167
WOMEN'S PETITE        66
EXCLUDE               52
Name: Gender, dtype: int64

# Prepare Data for Assortment Creation

Focus in creating assortments from 2 types of products ( top and pants) and not makable products
- this will help reduce obsolete inventory to create a new product

In [10]:
df_clean = df_clean.loc[(df_clean.loc[:,'Type']=='PANT') | (df_clean.loc[:,'Type']=='TOP')]

In [11]:
#uncomment filter needed
make = 'NOT MAKEABLE'
#make = 'MAKEABLE'
df_not_make = df_clean.loc[df_clean.loc[:,'Active']==make]

## Assortment Creation

In [12]:
# define list of columns to determine creation of assortments
determine_fields_column_names = ['Brand','ColorName','Gender','Size']
determine_fields_column_names

['Brand', 'ColorName', 'Gender', 'Size']

In [13]:
# define variable amoung which the different products an assortment is made of 
base_assortment_field = ['Type']
base_assortment_field

['Type']

In [14]:
# define quantifiable variable
amount = ['On Hand']

In [15]:
df_group_min = assortment_creation(df_not_make, determine_fields_column_names, base_assortment_field, amount)
df_group_min

,Brand,ColorName,Gender,Size,On Hand
0,Brand 1,color 1,WOMEN,L,2
1,Brand 1,color 1,WOMEN,XL,1
2,Brand 1,color 1094,WOMEN,S,2
3,Brand 1,color 111,WOMEN,S,7
4,Brand 1,color 3,WOMEN,2XL,1
...,...,...,...,...,...
1708,Brand 9,color 5953,WOMEN,M,160
1709,Brand 9,color 5953,WOMEN,S,106
1710,Brand 9,color 5953,WOMEN,XL,62
1711,Brand 9,color 5953,WOMEN,XS,139


## Reshape Assortments table and filter for possible amounts and Calculate Total

In [16]:
#pivot table to  to sort by sizes and fill blanks
df_group_min_pt = pd.pivot_table(df_group_min,index=['Brand','ColorName','Gender'], columns=['Size'], fill_value = 0)
# drop level
df_group_min_pt.columns = df_group_min_pt.columns.droplevel(0)

# sort the sizes
sizes = ['XXS', 'XS', 'S', 'M', 'L', 'XL', '2XL', '3XL', '4XL', '5XL']
df_group_min_pt = df_group_min_pt.loc[:,sizes]

# calculate total number of sets accross all sizes
df_group_min_pt['Total'] = df_group_min_pt.iloc[:,:].sum(axis=1).values

# filter out products with out sets
df_group_min_pt = df_group_min_pt[df_group_min_pt['Total']!=0]
# remove size hiarchy
df_group_min_pt.reset_index(inplace=True)

# id to match with other tables
df_group_min_pt['ID'] = df_group_min_pt['Brand'].str.strip() + df_group_min_pt['ColorName'].str.strip() + df_group_min_pt['Gender'].str.strip()
cols = ['ID','Brand',
 'ColorName',
 'Gender',
 'XXS',
 'XS',
 'S',
 'M',
 'L',
 'XL',
 '2XL',
 '3XL',
 '4XL',
 '5XL',
 'Total']
df_group_min_pt = df_group_min_pt.loc[:,cols]
df_group_min_pt

Size,ID,Brand,ColorName,Gender,XXS,XS,S,M,L,XL,2XL,3XL,4XL,5XL,Total
0,Brand 1color 1WOMEN,Brand 1,color 1,WOMEN,0,0,0,0,2,1,0,0,0,0,3
1,Brand 1color 1094WOMEN,Brand 1,color 1094,WOMEN,0,0,2,0,0,0,0,0,0,0,2
2,Brand 1color 111WOMEN,Brand 1,color 111,WOMEN,0,0,7,0,0,0,0,0,0,0,7
3,Brand 1color 3WOMEN,Brand 1,color 3,WOMEN,0,0,0,5,0,0,1,0,0,0,6
4,Brand 1color 4WOMEN,Brand 1,color 4,WOMEN,0,0,0,0,4,9,4,0,0,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Brand 9color 24WOMEN,Brand 9,color 24,WOMEN,0,0,0,0,0,1,0,0,0,0,1
356,Brand 9color 4954WOMEN,Brand 9,color 4954,WOMEN,0,0,0,0,0,0,0,8,0,0,8
357,Brand 9color 4957WOMEN,Brand 9,color 4957,WOMEN,0,0,0,0,0,0,0,27,0,0,27
358,Brand 9color 4959WOMEN,Brand 9,color 4959,WOMEN,19,0,0,0,0,0,0,9,0,0,28


### Find the number of continous sets for each product combination

In [17]:
df_group_min_pt = longest_streak_counter(df_group_min_pt, 4,14)
df_group_min_pt

Size,ID,Brand,ColorName,Gender,XXS,XS,S,M,L,XL,2XL,3XL,4XL,5XL,Total,NumOfContinous
0,Brand 1color 1WOMEN,Brand 1,color 1,WOMEN,0,0,0,0,2,1,0,0,0,0,3,2
1,Brand 1color 1094WOMEN,Brand 1,color 1094,WOMEN,0,0,2,0,0,0,0,0,0,0,2,1
2,Brand 1color 111WOMEN,Brand 1,color 111,WOMEN,0,0,7,0,0,0,0,0,0,0,7,1
3,Brand 1color 3WOMEN,Brand 1,color 3,WOMEN,0,0,0,5,0,0,1,0,0,0,6,1
4,Brand 1color 4WOMEN,Brand 1,color 4,WOMEN,0,0,0,0,4,9,4,0,0,0,17,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Brand 9color 24WOMEN,Brand 9,color 24,WOMEN,0,0,0,0,0,1,0,0,0,0,1,1
356,Brand 9color 4954WOMEN,Brand 9,color 4954,WOMEN,0,0,0,0,0,0,0,8,0,0,8,1
357,Brand 9color 4957WOMEN,Brand 9,color 4957,WOMEN,0,0,0,0,0,0,0,27,0,0,27,1
358,Brand 9color 4959WOMEN,Brand 9,color 4959,WOMEN,19,0,0,0,0,0,0,9,0,0,28,1


In [18]:
# export
#df_group_min_pt.to_csv(r"path"+date+".csv", encoding='utf-8', index=False)

# Filter for the products where a set can be created

In [19]:
df_clean['ID'] = df_clean['Brand'].str.strip() + df_clean['ColorName'].str.strip() + df_clean['Gender'].str.strip()
df_clean['ID_MC'] = df_clean['Model'].str.strip() + df_clean['Color'].str.strip()

In [20]:
df_clean = df_clean.loc[:,df_clean.columns[np.r_[25:27,0:18,24]]]

In [21]:
df_clean_set_components = df_clean.loc[df_clean.loc[:, 'ID'].isin(df_group_min_pt['ID'])]

In [22]:
df_f = pd.pivot_table(df_clean_set_components,index=['ID','ID_MC',
 'Division',
 'Brand',
 'ColorName',
 'Gender',
 'Model',
 'Color',
 'Type',
 'Description',
 'Price', 'Cost',
 'Status'], columns=['Size'], aggfunc='sum', fill_value = 0)
df_f.columns = df_f.columns.droplevel(0)
df_f = df_f.loc[:,sizes]
df_f['Total'] = df_f.iloc[:,:].sum(axis=1).values
df_f = df_f[df_f['Total']>0]
df_group_min_pt[['ID','NumOfContinous']]
df_f.reset_index(inplace=True)

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_15904\2483019731.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_f = pd.pivot_table(df_clean_set_components,index=['ID','ID_MC',


In [24]:
df_f = df_f.merge(df_group_min_pt[['ID','NumOfContinous']], on='ID', how='left')
df_f

Size,ID,ID_MC,Division,Brand,ColorName,Gender,Model,Color,Type,Description,...,M,L,XL,2XL,3XL,4XL,5XL,Total,NumOfContinous_x,NumOfContinous_y
0,Brand 11color 1270WOMEN,Model 1716DKAW,division 1,Brand 11,color 1270,WOMEN,Model 1716,DKAW,PANT,description 6,...,0,0,0,0,0,0,0,2,1,1
1,Brand 11color 1270WOMEN,Model 1716DKAW,division 1,Brand 11,color 1270,WOMEN,Model 1716,DKAW,PANT,description 6,...,0,0,33,0,0,0,0,33,1,1
2,Brand 11color 1270WOMEN,Model 1716DKAW,division 1,Brand 11,color 1270,WOMEN,Model 1716,DKAW,PANT,description 6,...,0,0,0,0,0,0,0,26,1,1
3,Brand 11color 1270WOMEN,Model 1716DKAW,division 1,Brand 11,color 1270,WOMEN,Model 1716,DKAW,PANT,description 6,...,16,0,0,0,0,0,0,16,1,1
4,Brand 11color 1270WOMEN,Model 1716DKAW,division 1,Brand 11,color 1270,WOMEN,Model 1716,DKAW,PANT,description 6,...,0,3,0,0,0,0,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11922,Brand 9color 5953WOMEN,Model 3845PZZZ,division 1,Brand 9,color 5953,WOMEN,Model 3845,PZZZ,TOP,description 139,...,0,120,0,0,0,0,0,120,8,8
11923,Brand 9color 5953WOMEN,Model 3845PZZZ,division 1,Brand 9,color 5953,WOMEN,Model 3845,PZZZ,TOP,description 139,...,0,0,0,0,0,0,0,139,8,8
11924,Brand 9color 5953WOMEN,Model 3845PZZZ,division 1,Brand 9,color 5953,WOMEN,Model 3845,PZZZ,TOP,description 139,...,160,0,0,0,0,0,0,160,8,8
11925,Brand 9color 5953WOMEN,Model 3845PZZZ,division 1,Brand 9,color 5953,WOMEN,Model 3845,PZZZ,TOP,description 139,...,0,0,0,0,0,0,0,106,8,8
